# VacationPy
----


In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv(os.path.join("..","output","cities.csv"))
del df["Unnamed: 0"]
df

,City Name,Country Code,Date (UTC),Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,Avarua,CK,2020-04-12 01:18:41,-21.21,-159.78,84.20,89,96,5.82
1,Hasaki,JP,2020-04-12 01:18:53,35.73,140.83,54.34,76,75,11.41
2,Puerto Ayora,EC,2020-04-12 01:18:53,-0.74,-90.35,80.87,74,75,11.41
3,Kangaba,ML,2020-04-12 01:18:53,11.93,-8.42,85.75,39,99,4.12
4,Kalat,PK,2020-04-12 01:18:53,29.02,66.59,59.18,26,48,4.65
...,...,...,...,...,...,...,...,...,...
580,Yazman,PK,2020-04-12 01:21:29,29.12,71.74,83.37,30,80,2.15
581,Sonoita,MX,2020-04-12 01:21:29,31.85,-112.83,62.01,53,81,10.36
582,Toro,CO,2020-04-12 01:21:29,4.61,-76.08,70.61,68,73,2.51
583,Dudinka,RU,2020-04-12 01:20:13,69.41,86.18,33.80,98,75,33.55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = df["Humidity (%)"].astype(float)

In [4]:
gmaps.configure(api_key = g_key)

fig = gmaps.figure(zoom_level=2,center=(0,0))

# Plot Heatmap
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = df.loc[(df["Temperature (F)"]>=68.00) & (df["Temperature (F)"]<=78.00) & (df["Cloudiness (%)"] <= 5) & (df["Wind Speed (MPH)"] <= 10.00)]
narrowed_city_df


,City Name,Country Code,Date (UTC),Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
70,Ambilobe,MG,2020-04-12 01:16:41,-13.20,49.05,72.12,84,0,3.96
118,Belgaum,IN,2020-04-12 01:19:21,15.87,74.50,68.00,94,0,2.17
157,Tura,IN,2020-04-12 01:19:33,25.52,90.22,71.02,26,0,2.80
160,Linhares,BR,2020-04-12 01:19:33,-19.39,-40.07,72.95,93,2,3.00
245,Santa Cruz,BO,2020-04-12 01:18:43,-17.80,-63.17,69.80,68,0,4.70
299,Kanakapura,IN,2020-04-12 01:18:15,12.55,77.42,72.77,48,0,3.04
320,Mulbagal,IN,2020-04-12 01:20:13,13.17,78.40,68.85,53,0,1.43
418,Jashpurnagar,IN,2020-04-12 01:20:48,22.90,84.15,71.82,22,0,2.98
517,Opuwo,NaN,2020-04-12 01:21:15,-18.06,13.84,68.49,55,3,6.42
545,Beohari,IN,2020-04-12 01:21:17,24.05,81.38,75.79,21,0,5.23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "Hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in narrowed_city_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotels = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [18]:
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df

,City Name,Country Code,Date (UTC),Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Hotel Name
70,Ambilobe,MG,2020-04-12 01:16:41,-13.20,49.05,72.12,84,0,3.96,Hotel National
118,Belgaum,IN,2020-04-12 01:19:21,15.87,74.50,68.00,94,0,2.17,PAI RESORTS
157,Tura,IN,2020-04-12 01:19:33,25.52,90.22,71.02,26,0,2.80,Hotel Polo Orchid
160,Linhares,BR,2020-04-12 01:19:33,-19.39,-40.07,72.95,93,2,3.00,Best Western Linhares Design Hotel
245,Santa Cruz,BO,2020-04-12 01:18:43,-17.80,-63.17,69.80,68,0,4.70,Buganvillas Hotel Suites & Spa
299,Kanakapura,IN,2020-04-12 01:18:15,12.55,77.42,72.77,48,0,3.04,Amegundi Resort
320,Mulbagal,IN,2020-04-12 01:20:13,13.17,78.40,68.85,53,0,1.43,HOTEL SREE SARAVANA BHAVAN
418,Jashpurnagar,IN,2020-04-12 01:20:48,22.90,84.15,71.82,22,0,2.98,"Hotel Nirvana, Jashpur"
545,Beohari,IN,2020-04-12 01:21:17,24.05,81.38,75.79,21,0,5.23,J.P. Residency Hotel & Restaurant


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City Name}</dd>
<dt>Country Code</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Details: {name}" for name in hotel_info])
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))